In [70]:
import numpy as np
import pandas as pd
import pyodbc

from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sqlalchemy import create_engine
from xgboost import XGBClassifier

from similar_fun import convert_to_float, price_class, dummies_encoding, search_for_neighbors, sorted_nearest_indexes

In [ ]:
server = 'TEST_SERVER'
database = 'test_db'
conn = pyodbc.connect(
   'DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';Trusted_Connection=yes')
conn = create_engine('mssql+pyodbc://' + server + '/' + database + '?trusted_connection=yes&driver=SQL Server')

###  Download of specifications

query = f"""SELECT id_product, id_specification, im_specification_value
        FROM dm_vw.im_product_specifications  sp
        INNER JOIN (SELECT DISTINCT ProductId FROM smartpr.ec_prices) prices
        ON sp.id_product = prices.ProductId
        WHERE category == "Ноутбуки"
"""
data = pd.read_sql(query,conn)

### Download of price

query = f"""SELECT DISTINCT prices.ProductId, 
        FIRST_VALUE(Price) OVER(PARTITION BY ProductId ORDER BY lst.Date DESC)  Price
        FROM smartpr.ec_prices prices
        INNER JOIN smartpr.ec_price lst 
        ON lst.Id = prices.Id
        WHERE ProductId in (SELECT DISTINCT id_product FROM dm_vw.im_product_specifications 
                            WHERE category == "Ноутбуки") 
        AND Price > 0
""" 
prices = pd.read_sql(query,conn) 

In [ ]:
df = data.pivot(index='id_product', columns='id_specification', values='im_specification_value')
df.reset_index(inplace=True) 
df.rename(columns={'id_product': 'ProductId'}, inplace=True) 
df = df.merge(prices, on='ProductId', how='inner') 

In [29]:
df['Вес'] = df['Вес'].astype(str)
df['Высота'] = df['Высота'].astype(str)

In [33]:
### Data preprocessing for boosting

columns = df.columns 
n = round(df.shape[0] * 0.8) 
for col in columns:
    if df[col].dropna().shape[0] < n:
        df.drop(col, axis=1, inplace=True)


key_words = ['вес', 'диагональ', 'высота', 'толщина', 'ширина', 'емкость', 'память','кол-во', 'количество', 'объем'
            'мощность', 'диаметр', 'диаметр']

size = df.shape[0]
columns = df.columns
for col in columns:
    col_name = col.lower()
    for word in key_words:
        if (col_name.find(word) != -1):
            df[col] =  df[col].astype(str)
            df[col + '_' + 'to_float'] = df[col].apply(convert_to_float)
            if df[col + '_' + 'to_float'].sum() >= size*0.99:  
                df.loc[df[col + '_' + 'to_float']==1, col] = df.loc[df[col + '_' + 'to_float']==1, col].astype(float)
                df.loc[df[col + '_' + 'to_float']==0, col] = df.loc[df[col + '_' + 'to_float']==1, col].median()

df1 = df.copy() 

In [39]:
price_index = np.where(columns == 'Price')[0][0]
feature_cols = df.columns[1:price_index] 
cat_features = []
for col in feature_cols: 
    try: 
        if col == "Емкость SSD (Гб)": 
            df.loc[df[col].isnull(), col]  = 0 
        else:
            df.loc[df[col].isnull(), col]  = df[col].median()      
    except TypeError:
        df[col] = df[col].astype(str) 
        cat_features.append(col)

le = LabelEncoder()
for col in cat_features:
    df[col] = le.fit_transform(df[col]) 

In [ ]:
### Division into price classes for boosting

x_min = df['Price'].quantile(10/100) 
x_max = df['Price'].quantile(90/100)
k = 8
length = (x_max - x_min) / k
x = []
x.append(x_min)
for i in range(k):
    x.append(x[-1] + length)

df['price_class'] = df['Price'].apply(price_class, borders=x)

In [ ]:
### Training of boosting

features = df[feature_cols]
answers = df['price_class']
X = features.values 
Y = answers.values

n_splits = min(10, df['price_class'].value_counts().min())

cv = StratifiedKFold(n_splits=n_splits)
model = XGBClassifier(eval_metric='mlogloss', use_label_encoder=False) 
weights = []
accuracy_list = []

for train, test in cv.split(X, Y):
    model.fit(X[train], Y[train])
    weights.append(model.feature_importances_)
    Y_pred = model.predict(X[test])
    elem_accuracy = accuracy_score(Y[test], Y_pred)
    accuracy_list.append(elem_accuracy)

In [56]:
### Selecting the best iteration and then main features

best_indexes = np.argsort(accuracy_list)[::-1][:2]
mean_weights = np.mean(np.array(weights)[best_indexes], axis=0)
sorted_weights = sorted(mean_weights)[::-1]

sum_w = 0
end_ind = 0
for i in range(len(sorted_weights)):
    sum_w += sorted_weights[i]
    end_ind = i + 1
    if (sum_w >= 0.5) and (end_ind > 1):  
        break

main_indexes = np.argsort(mean_weights)[::-1][:end_ind] 
main_weights = mean_weights[main_indexes]

In [ ]:
### Weights of the main features (for dummies encoding)

relation_weights = relation_weights / main_weights[-1]


### Preprocessing for finding neighbors 

selected_cols = feature_cols[main_indexes]

In [66]:
### For dummies encoding

for col in selected_cols:
    df1[col] = df1[col].astype(str) 

if 'Производитель' in feature_cols:
    brand_col ='Производитель' 
elif 'Бренд' in feature_cols:
    brand_col = 'Бренд' 

In [67]:
###  If there is Apple, then it must be separated

apple_indexes = np.where(df1[brand_col] == 'Apple')[0]
if len(apple_indexes) > 0:
    selected_apple = df1[df1[brand_col] == 'Apple'].copy()
    df1.drop(apple_indexes, axis=0, inplace=True)

In [71]:
### Finding neighbors for all brands except Apple

features = dummies_encoding(df1, selected_cols, relation_weights)
closest_ind = search_for_neighbors(features)

items = df1['ProductId'].values
neighbors = items[sorted_nearest_indexes(df1, brand_col, closest_ind, apple=False)]

n = neighbors.shape[0]
dct_neighbors = {items[i]: neighbors[i] for i in range(n)}

In [68]:
###  Finding neighbors for Apple if there it exists

if len(apple_indexes) > 0:
    features = dummies_encoding(selected_apple, selected_cols, relation_weights)
    closest_ind = search_for_neighbors(features)

    apple_items = selected_apple['ProductId'].values
    apple_neighbors = apple_items[sorted_nearest_indexes(selected_apple, brand_col, closest_ind, apple=True)]

    n = apple_neighbors.shape[0]
    dct_neighbors.update({apple_items[i]: apple_neighbors[i] for i in range(n)})